# Using `fmriprep` on the SLURM cluster



--------------------


#### HISTORY

* 7/9/2021 - dcosme revised for bbprime
* 5/22/20 - mbod updated for revised MURI fmriprep pipeline
* 1/22/20 - matt, jeesung and nicole initial test setup for using SLURM

In [1]:
import os

## 1. Create job files

* Each process that you send to SLURM needs a job file, which is a text file with the commands to be executed by SLURM

* For the bbprime project these job files should be placed in:

```
/data00/projects/geoscan_v2/scripts/FMRIPREP/jobs
```

* A job file has:

    1. some SLURM directives in lines beginning `#SBATCH`
    2. one or more commands to be run (prefixed with `srun`)
    
```
#!/bin/bash
#SBATCH --job-name=fmriprep_{ID}.job
#SBATCH --output=out/fmriprep_{ID}.out
#SBATCH --error=out/fmriprep_{ID}.err
#SBATCH --time=2-00:00

srun singularity run --cleanenv \
    -B /data00/tools/freesurfer/license.txt:/opt/freesurfer/license.txt  \
    -B /data00/projects/geoscan_v2/data/bids_data:/data \
    -B /data00/projects/geoscan_v2/data/bids_data/derivatives:/out  \
    -B /data00/projects/geoscan_v2/data/bids_data/derivatives/working:/work \
    /data00/tools/singularity_images/fmriprep_v20.0.6 /data /out -w /work \
    --ignore slicetiming \
    --nthread 8 \
    participant --participant_label {ID} \
    --skip-bids-validation
```

### Define variables

In [2]:
project = 'geoscan_v2'
project_dir = os.path.join('/data00/projects', project)
slurm_dir = os.path.join(project_dir, 'scripts/FMRIPREP/jobs')
os.makedirs(slurm_dir, exist_ok=True)

### Create job template

OLD VERSION: Fieldmap correction

In [3]:
job_file_template = r'''#!/bin/bash
#SBATCH --job-name=fmriprep_{ID}.job
#SBATCH --output=out/fmriprep_{ID}.out
#SBATCH --error=out/fmriprep_{ID}.err
#SBATCH --time=2-00:00

srun singularity run --cleanenv \
    -B /data00/tools/freesurfer/license.txt:/opt/freesurfer/license.txt  \
    -B /data00/projects/geoscan_v2/data/bids_data:/data \
    -B /data00/projects/geoscan_v2/data/bids_data/derivatives:/out  \
    -B /data00/projects/geoscan_v2/data/bids_data/derivatives/working:/work \
    /data00/tools/singularity_images/fmriprep_v20.0.6 /data /out -w /work \
    --ignore slicetiming \
    --nthread 8 \
    participant --participant_label {ID} \
    --skip-bids-validation
'''

NEW VERSION: Fieldmap-less correction

In [4]:
job_file_template = r'''#!/bin/bash
#SBATCH --job-name=fmriprep-syn_{ID}.job
#SBATCH --output=out/fmriprep-syn_{ID}.out
#SBATCH --error=out/fmriprep-syn_{ID}.err
#SBATCH --time=2-00:00

srun singularity run --cleanenv \
    -B /data00/tools/freesurfer/license.txt:/opt/freesurfer/license.txt  \
    -B /data00/projects/geoscan_v2/data/bids_data:/data \
    -B /data00/projects/geoscan_v2/data/bids_data/derivatives:/out  \
    -B /data00/projects/geoscan_v2/data/bids_data/derivatives/working:/work \
    /data00/tools/singularity_images/fmriprep_v20.0.6 /data /out -w /work \
    --ignore slicetiming fieldmaps \
    --use-syn-sdc \
    --nthread 8 \
    participant --participant_label {ID} \
    --skip-bids-validation
'''

In [4]:
print(job_file_template.format(ID='sub-GS005'))

#!/bin/bash
#SBATCH --job-name=fmriprep-syn_sub-GS005.job
#SBATCH --output=out/fmriprep-syn_sub-GS005.out
#SBATCH --error=out/fmriprep-syn_sub-GS005.err
#SBATCH --time=2-00:00

srun singularity run --cleanenv \
    -B /data00/tools/freesurfer/license.txt:/opt/freesurfer/license.txt  \
    -B /data00/projects/geoscan_v2/data/bids_data:/data \
    -B /data00/projects/geoscan_v2/data/bids_data/derivatives:/out  \
    -B /data00/projects/geoscan_v2/data/bids_data/derivatives/working:/work \
    /data00/tools/singularity_images/fmriprep_v20.0.6 /data /out -w /work \
    --ignore slicetiming fieldmaps \
    --use-syn-sdc \
    --nthread 8 \
    participant --participant_label sub-GS005 \
    --skip-bids-validation



### Identify participants who lack job files
Of the participants listed in the `{project_dir}/data/bids_data`, identify those who lack a job file and create it

In [7]:
subs = [s for s in os.listdir(os.path.join(project_dir, 'data/bids_data'))
                       if s.startswith('sub-GEO')]

existing_job_files = [s for s in os.listdir(slurm_dir)
                       if s.endswith('.job') and s.count('sub-GS')==1]

subs_to_process = [sub for sub in subs if 'fmriprep-syn_' + sub + '.job' not in existing_job_files]

In [8]:
subs_to_process 

['sub-GEO015',
 'sub-GEO021',
 'sub-GEO033',
 'sub-GEO036',
 'sub-GEO037',
 'sub-GEO053',
 'sub-GEO061',
 'sub-GEO064',
 'sub-GEO067',
 'sub-GEO068',
 'sub-GEO070',
 'sub-GEO071',
 'sub-GEO073',
 'sub-GEO074',
 'sub-GEO078']

### Create a job file for each participant in `subs_to_process`

In [9]:
for s in subs_to_process:
    file_path = os.path.join(slurm_dir, 'fmriprep-syn_{}.job').format(s)
    print('Creating', file_path)

    with open(file_path.format(s), 'w') as job:
        job.write(job_file_template.format(ID=s))

Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO015.job
Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO021.job
Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO033.job
Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO036.job
Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO037.job
Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO053.job
Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO061.job
Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO064.job
Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO067.job
Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO068.job
Creating /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs/fmriprep-syn_sub-GEO070.job
Creating /data00/projects/geoscan_v2/script

## 2. Submit jobs to SLURM cluster

### How to log onto the SLURM master


* master name: `cls000.asc.upenn.edu`


```
ssh <JANUS_UN>@asc.upenn.edu@cls000
```


* This will give you a terminal on the SLURM master node where you can look at the process queue and schedule jobs.

### How to schedule a job to SLURM

* Use the `sbatch` command

    ```
    sbatch -D /data00/slurm_jobs/slurm_bbprime -c 8 name_of_job_file
    
    ```

    * `-D` is the working directory
    * `-c` the number of processors to use
    * then the name of the job file

### Create log file directory

If this doesn't exist, your job will fail without explanation

In [10]:
os.makedirs(os.path.join(slurm_dir, 'out'), exist_ok=True)

### Run all jobs
To submit all the jobs, copy and past the following lines of code into the terminal after logging into the slurm cluster

In [11]:
print("Alternatively, submit the whole batch by copying and pasting the following lines in terminal:")
print("")

print(f"cd {slurm_dir}")
for s in subs_to_process:
    print(f"sbatch -D {slurm_dir} -c 8 fmriprep-syn_{s}.job")
print(" ")

Alternatively, submit the whole batch by copying and pasting the following lines in terminal:

cd /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs
sbatch -D /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs -c 8 fmriprep-syn_sub-GEO015.job
sbatch -D /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs -c 8 fmriprep-syn_sub-GEO021.job
sbatch -D /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs -c 8 fmriprep-syn_sub-GEO033.job
sbatch -D /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs -c 8 fmriprep-syn_sub-GEO036.job
sbatch -D /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs -c 8 fmriprep-syn_sub-GEO037.job
sbatch -D /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs -c 8 fmriprep-syn_sub-GEO053.job
sbatch -D /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs -c 8 fmriprep-syn_sub-GEO061.job
sbatch -D /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs -c 8 fmriprep-syn_sub-GEO064.job
sbatch -D /data00/projects/geoscan_v2/scripts/FMRIPREP/jobs -c 8 fmriprep-syn_sub-GEO067.job
sbatch -D /data